# Using FalkorGraphRagCapability with agents for GraphRAG Question & Answering

AG2 provides GraphRAG integration using agent capabilities. This is an example to integrate FalkorDB (a Knowledge Graph database).

## Install FalkorDB SDK

In [ ]:
%pip install graphrag_sdk==0.3.3

## Set Configuration and OpenAI API Key

In order to use FalkorDB you need to have an OpenAI key in your environment variable `OPENAI_API_KEY`.

You can utilise an OAI_CONFIG_LIST file and extract the OpenAI API key and put it in the environment, as will be shown in the following cell.

Alternatively, you can load the environment variable yourself.

````{=mdx}
:::tip
Learn more about configuring LLMs for agents [here](/docs/topics/llm_configuration).
:::
````

In [1]:
import os

import autogen

config_list = autogen.config_list_from_json(env_or_file="OAI_CONFIG_LIST")

# Put the OpenAI API key into the environment
os.environ["OPENAI_API_KEY"] = config_list[0]["api_key"]

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


````{=mdx}
:::important
The default model for loading graph data and answering questions using FalkorDB's SDK is OpenAI's GPT 4o and this can be changed by setting the `model` parameter on the FalkorGraphQueryEngine.
:::
````

## Create a Knowledge Graph with Your Own Data

**Note:** You need to have a FalkorDB graph database running. If you are running one in a Docker container, please ensure your Docker network is setup to allow access to it.

In this example, the FalkorDB endpoint is set to host="192.168.0.1" and port=6379, please adjust accordingly. For how to set up FalkorDB, please refer to https://docs.falkordb.com/


Below, we have some sample data from IMDB on the movie 'The Matrix'.

We define the entities and attributes that we want in the graph database (`Actor` with `name` and `Movie` with `title`) and the relationships (Actors `ACTED` in Movies). This allows your RAG agent to answer questions about actors in the movie.

We then initialise the database with that text document, creating the graph in FalkorDB.

In [ ]:
from graphrag_sdk import Attribute, AttributeType, Entity, Ontology, Relation

from autogen.agentchat.contrib.graph_rag.document import Document, DocumentType
from autogen.agentchat.contrib.graph_rag.falkor_graph_query_engine import FalkorGraphQueryEngine

# Auto generate graph schema from unstructured data
input_path = "../test/agentchat/contrib/graph_rag/the_matrix.txt"

movie_ontology = Ontology()
movie_ontology.add_entity(
    Entity(label="Actor", attributes=[Attribute(name="name", attr_type=AttributeType.STRING, unique=True)])
)
movie_ontology.add_entity(
    Entity(label="Movie", attributes=[Attribute(name="title", attr_type=AttributeType.STRING, unique=True)])
)
movie_ontology.add_relation(Relation(label="ACTED", source="Actor", target="Movie"))

query_engine = FalkorGraphQueryEngine(
    name="IMDB",
    host="192.168.0.1",  # Change
    port=6379,  # if needed
    ontology=movie_ontology,
)

input_documents = [Document(doctype=DocumentType.TEXT, path_or_url=input_path)]

query_engine.init_db(input_doc=input_documents)

## Add capability to a ConversableAgent and query them

With FalkorDB setup, we can now associate the GraphRAG capability with a ConversableAgent and have a chat with it.

In [3]:
from autogen import ConversableAgent, UserProxyAgent
from autogen.agentchat.contrib.graph_rag.falkor_graph_rag_capability import FalkorGraphRagCapability

# Create a ConversableAgent (no LLM configuration)
graph_rag_agent = ConversableAgent(
    name="matrix_agent",
    human_input_mode="NEVER",
)

# Associate the capability with the agent
graph_rag_capability = FalkorGraphRagCapability(query_engine)
graph_rag_capability.add_to_agent(graph_rag_agent)

# Create a user proxy agent to converse with our RAG agent
user_proxy = UserProxyAgent(
    name="user_proxy",
    code_execution_config=False,
    is_termination_msg=lambda msg: "TERMINATE" in msg["content"],
    human_input_mode="ALWAYS",
)

user_proxy.initiate_chat(graph_rag_agent, message="Name a few actors who've played in 'The Matrix'")

# You will be prompted, as a human in the loop, after the response - feel free to ask more questions.

user_proxy (to matrix_agent):

Name a few actors who've played in 'The Matrix'

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

Keanu Reeves, Laurence Fishburne, and Carrie-Anne Moss are a few actors who've played in 'The Matrix'.

--------------------------------------------------------------------------------
user_proxy (to matrix_agent):

Anyone else?

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

Hugo Weaving, Lilly Wachowski, and Lana Wachowski are other individuals associated with 'The Matrix'.

--------------------------------------------------------------------------------
user_proxy (to matrix_agent):

Anyone else?

--------------------------------------------------------------------------------
matrix_agent (to user_proxy):

No, there are no other actors associated with 'The Matrix' besides those already mentioned.

--------------------------------

ChatResult(chat_id=None, chat_history=[{'content': "Name a few actors who've played in 'The Matrix'", 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Keanu Reeves, Laurence Fishburne, and Carrie-Anne Moss are a few actors who've played in 'The Matrix'.", 'role': 'user', 'name': 'matrix_agent'}, {'content': 'Anyone else?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "Hugo Weaving, Lilly Wachowski, and Lana Wachowski are other individuals associated with 'The Matrix'.", 'role': 'user', 'name': 'matrix_agent'}, {'content': 'Anyone else?', 'role': 'assistant', 'name': 'user_proxy'}, {'content': "No, there are no other actors associated with 'The Matrix' besides those already mentioned.", 'role': 'user', 'name': 'matrix_agent'}], summary="No, there are no other actors associated with 'The Matrix' besides those already mentioned.", cost={'usage_including_cached_inference': {'total_cost': 0}, 'usage_excluding_cached_inference': {'total_cost': 0}}, human_input=['Anyone els